In [14]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import torch as torch
import os

In [2]:
DATA_DIRECTORY = "../datos/"

In [3]:
diagnosticos  = pd.read_excel(DATA_DIRECTORY+"RESUMEN TAC CEREBRALES.xlsx")

In [18]:
# diccionario cuya llave es el id de paciente y el valor una lista 
# donde cada elemento de la lista es la matriz de una i
diccionario_imagenes_pacientes = dict()

for paciente in diagnosticos.paciente:
    directorio_paciente = DATA_DIRECTORY+"paciente_"+str(paciente)
    archivos_paciente = os.listdir(directorio_paciente)
    
    lista_imagenes_paciente = []
    for archivo in archivos_paciente:
        if archivo.endswith(".jpg"):
            imagen = mpimg.imread(directorio_paciente+"/"+archivo)
            lista_imagenes_paciente.append(imagen)
            
    diccionario_imagenes_pacientes[paciente] = lista_imagenes_paciente
    

In [35]:
EPOCHS = 3

In [48]:

for epoch in range(EPOCHS):
    # en cada epoch procesar los pacientes en orden aleatorio
    pacientes = np.random.choice(np.array(diagnosticos.paciente),size= len(diagnosticos.paciente),replace=False)
    
    for paciente in pacientes:
        imagenes_paciente = diccionario_imagenes_pacientes.get(paciente)
        
        indices_imagenes_pacientes = np.arange(0,len(imagenes_paciente)-1,step=1)
        indices_aleatorios_imagenes = np.random.choice(indices_imagenes_pacientes,len(indices_imagenes_pacientes),replace=False)
        
        for indice in indices_aleatorios_imagenes:
            imagen_paciente = imagenes_paciente[indice]
            
            print("Alimentando paciente {} e imagen {} al modelo".format(paciente,indice))

Alimentando paciente 5 e imagen 3 al modelo
Alimentando paciente 5 e imagen 11 al modelo
Alimentando paciente 5 e imagen 0 al modelo
Alimentando paciente 5 e imagen 12 al modelo
Alimentando paciente 5 e imagen 4 al modelo
Alimentando paciente 5 e imagen 6 al modelo
Alimentando paciente 5 e imagen 2 al modelo
Alimentando paciente 5 e imagen 7 al modelo
Alimentando paciente 5 e imagen 10 al modelo
Alimentando paciente 5 e imagen 5 al modelo
Alimentando paciente 5 e imagen 13 al modelo
Alimentando paciente 5 e imagen 8 al modelo
Alimentando paciente 5 e imagen 1 al modelo
Alimentando paciente 5 e imagen 9 al modelo
Alimentando paciente 4 e imagen 3 al modelo
Alimentando paciente 4 e imagen 9 al modelo
Alimentando paciente 4 e imagen 7 al modelo
Alimentando paciente 4 e imagen 12 al modelo
Alimentando paciente 4 e imagen 4 al modelo
Alimentando paciente 4 e imagen 10 al modelo
Alimentando paciente 4 e imagen 8 al modelo
Alimentando paciente 4 e imagen 6 al modelo
Alimentando paciente 4 e i